# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [78]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #for line in lines:
    #    for x1,y1,x2,y2 in line:
    #        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    left_lines = []
    right_lines = []

    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if (slope > 0.2 and slope <0.95):
                left_lines.append([x1,y1,x2,y2,slope])
            elif (slope < -0.2 and slope > -0.95):
                right_lines.append([x1,y1,x2,y2,slope])
    
    imshape = img.shape

    #Right line
    right_cor = []
    right_cor = np.mean(right_lines, axis=0)
    if right_cor.size>1:
        x1,y1,x2,y2,slope = right_cor
        b = y1 - slope*x1
        y1_n = imshape[1]
        x1_n = (y1_n - b)/slope
        y2_n = (imshape[0])*3/5
        x2_n = (y2_n - b)/slope
        cv2.line(img,(int(x1_n),int(y1_n)),(int(x2_n),int(y2_n)),(255,0,0),10)

    #Left line
    left_cor = np.mean(left_lines, axis=0)
    if left_cor.size>1:
        x1,y1,x2,y2,slope = np.mean(left_lines, axis=0)
        b = y1 - slope*x1
        y1_n = imshape[1]
        x1_n = (y1_n - b)/slope
        y2_n = (imshape[0])*3/5
        x2_n = (y2_n - b)/slope
        cv2.line(img,(int(x1_n),int(y1_n)),(int(x2_n),int(y2_n)),(255,0,0),10)       
    
    return img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [58]:
import os
list = os.listdir("test_images/")


## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [69]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

for i in range(len(list)):
    if list[i]==".DS_Store":
        continue
    else:
        #Read image from list
        image = mpimg.imread('test_images/{}'.format(list[i]))

        #Convert the image to grayscale
        gray = grayscale(image)

        #Define a kernel size and apply Gaussian smoothing
        kernel_size = 5
        blur_gray = gaussian_blur(gray, kernel_size)

        #Define parameter for Canny edge and apply
        low_threshold = 50
        high_threshold = 150
        edges = canny(blur_gray, low_threshold, high_threshold)

        #Define region of interest and apply to edges
        imshape = image.shape
        #vertices = np.array([[(0,imshape[0]),(450, 325), (510, 325), (imshape[1],imshape[0])]], dtype=np.int32)
        vertices = np.array([[(imshape[1]*0.05,imshape[0]),(imshape[1]*0.46, (imshape[0])*3/5), (imshape[1]*0.54, (imshape[0])*3/5), (imshape[1]*0.95,imshape[0])]], dtype=np.int32)

        masked_image = region_of_interest(edges,vertices)

        #Define the Hough transform parameters

        rho = 1 # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 35   # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 40 #minimum number of pixels making up a line
        max_line_gap = 50 # maximum gap in pixels between connectable line segments

        line_img = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)

        new_img = weighted_img(line_img,image)
        #plt.figure()
        #plt.imshow(masked_image)

        cv2.imwrite('test_images_output/{}'.format(list[i]), cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR))


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [48]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    #Convert the image to grayscale
    gray = grayscale(image)

    #Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    #Define parameter for Canny edge and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    #Define region of interest and apply to edges
    imshape = image.shape
    #vertices = np.array([[(0,imshape[0]),(450, 325), (510, 325), (imshape[1],imshape[0])]], dtype=np.int32)
    vertices = np.array([[(imshape[1]*0.05,imshape[0]),(imshape[1]*0.46, (imshape[0])*3/5), (imshape[1]*0.54, (imshape[0])*3/5), (imshape[1]*0.95,imshape[0])]], dtype=np.int32)




    masked_image = region_of_interest(edges,vertices)

    #Define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 35   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 50 # maximum gap in pixels between connectable line segments

    line_img = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)

    return weighted_img(line_img,image)

Let's try the one with the solid white lane on the right first ...

In [49]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   4%|▍         | 9/221 [00:00<00:02, 88.70it/s, now=None]

0.6152625444233155
Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4

0.6152625444233155
0.6198488623221192
0.6237403174587539
0.6206731361980031
0.6158525978138037
0.6133296100981176
0.6132629907395027
0.613398023307436
0.6154774442268894
0.6166178463579391
0.625811405838135
0.62591457595243
0.6361330607249825
0.6379479339964206
0.6404829257056556


t:  12%|█▏        | 27/221 [00:00<00:02, 86.90it/s, now=None]

0.6440631936160005
0.6559733599950734
0.6442044078108672
0.6430438386465557
0.6406640529128976
0.6405773464129698
0.6402021720363817
0.6371488737877803
0.6436355633028118
0.6430123307635421
0.6392296604261498
0.6405133825019272
0.6456300663908036
0.6515881996684942
0.6387840389186139
0.6377360186340966
0.6363569379070151
0.635403194170765


t:  20%|██        | 45/221 [00:00<00:02, 85.82it/s, now=None]

...

0.5735317344348779
0.5783674075773617
0.571542694585495
0.5689339849653409
0.5705642107729777
0.5687648396582835
0.5682678133800408
0.5707027097456233


t:  96%|█████████▋| 213/221 [00:05<00:00, 32.29it/s, now=None]

0.5645353597793759
0.5662088587011178
0.5618876730059142
0.5594454234785422
0.5680077984634504
0.5688701300811876
0.5656054094013152
0.5659293920224433


0.5660419171518654
0.5645186152397604
0.5665674910464698
0.5677605207742824
0.5682885857655743


Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
CPU times: user 2.7 s, sys: 555 ms, total: 3.26 s
Wall time: 6.75 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [77]:
challenge_output = 'test_videos_output/20160330_184352_NF.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/20160330_184352_NF.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

                                                             
                                                          

t:  39%|███▉      | 99/251 [27:07<00:09, 16.54it/s, now=None]

                                                             
                                                          

t:  39%|███▉      | 99/251 [27:07<00:09, 16.54it/s, now=None]

t:  39%|███▉      | 99/251 [21:23<00:10, 14.26it/s, now=None]


t:   0%|          | 0/1830 [00:00<?, ?it/s, now=None]


t:   0%|          | 4/1830 [00:00<00:52, 35.05it/s, now=None]

5
5
Moviepy - Building video test_videos_output/20160330_184352_NF.mp4.
Moviepy - Writing video test_videos_output/20160330_184352_NF.mp4

5
5
5
5
5
5





t:   0%|          | 8/1830 [00:00<00:55, 32.77it/s, now=None]

5
5
5
5
5
5
5
5
5
5
5
5
5
5





t:   1%|          | 12/1830 [00:00<00:54, 33.21it/s, now=None]


t:   1%|          | 16/1830 [00:00<00:57, 31.77it/s, now=None]

...

5
5
5
5
5
5
5
5





t: 100%|██████████| 1830/1830 [01:53<00:00, 13.73it/s, now=None]


                                                                

5
5


                                                             
                                                          

t:  39%|███▉      | 99/251 [29:03<00:09, 16.54it/s, now=None]

                                                             
                                                          

t:  39%|███▉      | 99/251 [29:03<00:09, 16.54it/s, now=None]

t:  39%|███▉      | 99/251 [23:18<00:10, 14.26it/s, now=None]

Moviepy - Done !
Moviepy - video ready test_videos_output/20160330_184352_NF.mp4
CPU times: user 1min 8s, sys: 12.2 s, total: 1min 20s
Wall time: 1min 55s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))